# Lista 13 (8 pkt.)

We wszystkich poniższych zadaniach będziemy używać **OpenSSL** https://www.openssl.org/.

## Krótki tutorial:

### generowanie klucza prywatnego
```
openssl genpkey -algorithm RSA -pkeopt rsa_keygen_bits:2048 -pkeyopt rsa_keygen_pubexp:3 -out privkey.pem
```
```-algorithm``` wybiera algorytm do generacji klucza, tu ```RSA```

```-pkeyopt rsa_keygen_bits:2048``` ustala długość klucza, tu ```2048``` bitów

```-pkeyopt rsa_keygen_pubexp:3``` ustala wartość $e$, tu ```3```

```-out privkey.pem``` zapisuje klucz prywatny w pliku ```privkey.pem```
### generowanie klucza publicznego
```
openssl pkey -in privkey.pem -pubout -out pubkey.pem
```
```-in privkey.pem``` wczytuje plik z kluczem prywatnym

```-pubout``` ustala, że chcemy uzyskać klucz publiczny

```-out pubkey.pem``` zapisuje klucz publiczny w pliku ```pubkey.pem```

### Diffy-Helman
- generowanie publicznych parametrów $p$ i $g$
```
openssl genpkey -genparam -algorithm DH -out dh_params.pem
```
- generowanie klucza prywatnego $a$
```
openssl genpkey -paramfile dh_params.pem -out privkey.pem
```
- generowanie klucza publicznego $g^a$
```
openssl pkey -in privkey.pem -pubout -out pubkey.pem
```
- generowanie wspólnego sekretu $k=(g^b)^a=(g^a)^b$
```
openssl pkeyutl -derive -inkey privkey_A.pem -peerkey pubkey_B.pem -out secret.bin
```

### wyświetlanie w przyjaznej formie
- klucz prywatny
```
openssl pkey -in privkey.pem -text -noout
```
- klucz publiczny
```
openssl pkey -pubin -in pubkey.pem -text -noout
```
- certyfikat
```
openssl x509 -in user.crt -text -noout
```
- parametry Diffiego-Helmana
```
openssl pkeyparam -in dh_params.pem -text -noout
```
### generowanie żądania popisania certyfikatu
```
openssl req -new -key privkey.pem -out req.csr
```
```-key privkey.pem``` wskazuje plik z naszym kluczem prywatnym

```-out req.csr``` zapisuje żądanie certyfikatu do pliku ```req.csr``

po wpisaniu tej komendy pojawi się seria pytań o dane do certyfikatu, można wpisać jakieś rzeczy typu PL, Cracow, UJ itd., można też wpisać '.' (kropkę) i w ten sposób pominąć dane pole

Uwaga: aby certyfikat przechodził później weryfikację bez problemu, dane wpisane w certyfikat użytkownika, i dane wpisane w certyfikat **CA** nie mogą być identyczne.

### generowanie samopodpisanego certyfikatu przez **CA**
```
openssl req -x509 -new -nodes -key rootkey.pem -sha256 -days 1024 -out root.crt
```

```-key rootkey.pem``` wskazuje klucz prywatny **CA**

```-sha256``` wybiera funkcję hashującą używaną do podpisu, tu **SHA 256**

```-days 1024``` ustala termin ważności certyfikatu

```-out root.crt``` zapisuje certyfikat do pliku ```root.crt```

### wygenerowanie i podpisanie certyfiaktu przez **CA** na podstawie żądania
```
openssl x509 -req -in req.csr -CA root.crt -CAkey rootkey.pem -CAcreateserial -out user.crt -days 500 -sha256
```
```-req -in req.csr``` wskazuje żądanie

```-CA root.crt``` wskazuje klucz prywatny **CA**

```-CAcreateserial``` tworzy numer seryjny cerrtyfikatu

```-out user.crt``` zapisuje podpisany certyfikat do pliku ```user.crt```

```-days 500``` ustala termin ważności certyfikatu, tu ```500``` dni

```-sha256``` wybiera funkcję hashującą używaną do podpisu, tu **SHA 256**
### weryfikacja certyfikatu
```
openssl verify -CAfile root.crt user.crt
```
```-CAfile root.crt``` wskazuje certyfikat **CA** na podstawie którego weryfikujemy certyfiakt użytkownika

```user.crt``` certyfikat, który weryfikujemy
### wydobycie klucza publicznego użytkownika z jego certyfikatu
```
openssl x509 -pubkey -in user.crt -noout > pubkey.pem
```
```-pubkey``` wskazuje, że wyciągamy klucz publiczny

```-in user.crt``` wskazuje plik z certyfikatem

```> pubkey.pem``` zapisuje uzyskany klucz do pliku ```pubkey.pem```

### szyfrowanie pliku za pomocą szyfru asymetrycznego
```
openssl pkeyutl -encrypt -in file.txt -pubin -inkey pubkey.pem -out file.enc
```
```-encrypt``` włącza tryb szyfrowania

```-in file.txt``` wskazuje plik do zaszyfrowania

```-pubin -inkey pubkey.pem``` wskazuje klucz publiczny, którym szyfrujemy

```-out file.enc``` zapsiuje zaszyfrowany plik do pliku ```file.enc```
### deszyfrowanie pliku za pomocą szyfru asymetrycznego
```
openssl pkeyutl -decrypt -in file.enc -inkey privkey.pem -out file.txt
```
```-decrypt``` włącza tryb deszyfrowania

```-in file.enc``` wskazuje plik do deszyfrowania

```-inkey pubkey.pem``` wskazuje klucz prywatny, którym deszyfrujemy

```-out fil.txt``` zapsiuje odszyfrowany plik do pliku ```file.txt```

### szyfrowanie pliku za pomocą szyfru symetrycznego
- na podstawie hasła
```
openssl enc -aes-256-cbc -p -pbkdf2 -in file.txt -out file_enc.bin
```
pojawi się zapytanie o hasło, można też dodać ```-pass file:key_pass.pem``` aby pobrać hasło z pliku ```key_pass.pem```

```-aes-256-cbc``` wybiera jako szyfr **AES** z kluczem **256** bitowym w trybie pracy **CBC**

```-p``` wypisuje na ekranie klucz, sól i wektor inicjujący

```--pbkdf2``` ustala metodę używaną przy generowaniu klucza

```-in file.txt``` plik do zaszyfrowania

```file_enc.bin``` zaszyfrowany plik
- na podstawie klucza
```
openssl enc -aes-256-cbc -K XXXXXXXXX -iv XXXXXXXX -p -in file.txt -out file_enc.bin
```
```-aes-256-cbc``` wybiera jako szyfr **AES** z kluczem **256** bitowym w trybie pracy **CBC**

```-K XXXXXXXXX``` ustala klucz na ```XXXXXXXXX```, format szesnastkowy, odpowiednia długość do szyfru

```-iv XXXXXXXXX``` ustala wektor inicjujący na ```XXXXXXXXX```, format szesnastkowy, odpowiednia długość do szyfru

```-p``` wypisuje na ekranie klucz, sól i wektor inicjujący

```-in file.txt``` plik do zaszyfrowania

```file_enc.bin``` zaszyfrowany plik

### deszyfrowanie pliku za pomocą szyfru symetrycznego
- na podstawie hasła
```
openssl enc -aes-256-cbc -d -p -pbkdf2 -in file_enc.bin -out file.txt
```
pojawi się zapytanie o hasło, można też dodać ```-pass file:key_pass.pem``` aby pobrać hasło z pliku ```key_pass.pem```

```-aes-256-cbc``` wybiera jako szyfr **AES** z kluczem **256** bitowym w trybie pracy **CBC**

```-d``` wybiera tryb deszyfracji

```-pass file:key_pass.pem``` wybiera deszyfrowanie na podstawie klucza wygenerowanego z hasła znajdującego się w pliku ```key_pass.pem```

```-p``` wypisuje na ekranie klucz, sól i wektor inicjujący

```-pbkdf2``` ustala funkcję hashującą używaną przy generowaniu klucza

```-in file_enc.bin``` plik do deszyfrowania

```file.txt``` odszyfrowany plik
- na podstawie klucza
```
openssl enc -aes-256-cbc -d -K XXXXXXXXX -iv XXXXXXXX -p -in file.enc -out file.txt
```
```-aes-256-cbc``` wybiera jako szyfr **AES** z kluczem **256** bitowym w trybie pracy **CBC**

```-d``` wybiera tryb deszyfracji

```-K XXXXXXXXX``` ustala klucz na ```XXXXXXXXX```, format szesnastkowy, odpowiednia długość do szyfru

```-iv XXXXXXXXX``` ustala wektor inicjujący na ```XXXXXXXXX```, format szesnastkowy, odpowiednia długość do szyfru

```-p``` wypisuje na ekranie klucz, sól i wektor inicjujący

```-in file.enc``` plik do odszyfrowania

```file.txt``` odszyfrowany plik

### generowanie podpisu elektronicznego
```
openssl dgst -sha1 -sign privkey.pem -out signature.bin file.txt
```
```-sha1``` wybiera funkcję hashującą **SHA 1**

```-sign privkey.pem``` podpisuje kluczem prywatnym ```privkey.pem```

```-out signature.bin``` zapisuje podpis do pliku binarnego ```signature.bin```

```file.txt``` podpisywany plik
### weryfikacja podpisu elektronicznego
```
openssl dgst -sha1 -verify pubkey.pem -signature signature.bin file.txt
```
```-sha1``` wybiera funkcję hashującą **SHA 1**

```-verify pubkey.pem``` weryfikuje podpis kluczem publicznm ```pubkey.pem```

```-signature signature.bin``` wskazuje plik z podpisem do weryfikacji

```file.txt``` plik, którego podpis weryfikujemy

### generowanie pseudo-losowych bajtów
```
openssl rand -base64 32 -out symkey.pem
```
- ```-base64``` ustala kodowanie na base64

- ```32``` liczba bajtów do wygenerowania

- ```-out symkey.pem``` zapisuje ciąg bajtów do symkey.pem


## Zadanie 1 (1 pkt.)

Zdeszyfruj plik **AESencryptedCBC.enc** za pomocą klucza:

8541F781259ADA8631F18A9A8E97771BFE395CF7ACBE5F77

oraz wektora inicjującego:

99B54E58FF44F7599465E6888A3C6C1E

w trybie **CBC** z kluczem **192** bitowym.

## Zadanie 2 (1 pkt.)

Zaszyfruj plik **kamien.txt** za pomocą hasła w trybie **OFB** z kluczem **256** bitowym.

## Zadanie 3 (1 pkt.)

Stwórz dwa foldery, **Alice** oraz **Bob**, i przeprowadź protokół uzgadniania klucza za pomocą metody Diffiego-Helmana pomiędzy tymi folderami. Gdy uzsykasz w obu folderach plik **secret.bin**, który jest plikiem binarnym zawierającym uzgodniony ciąg bitów pomiędzy Alicją i Bobem, zaszyfruj plik **rozwijajac_rilkego.txt** w folderze Alicji podając jako hasło plik **secret.bin** a następnie zdeszyfruj go w folderze Boba za pomocą jego pliku **secret.bin**. Użyj AES 256 w dowolnym trybie.

## Zadanie 4 (1 pkt.)

Klikając w kłodkę w przeglądarce internetowej otwórz certyfikaty następujących stron:

- Wikipedia
- Google
- Strona tego kursu kryptografii

Odczytaj z tych certyfikatów następujące informacje:
- data ważności certyfikatu
- rodzaj używanego algorytmu asymetrycznego
- klucz publiczny
- rodzaj używanego algorytmu do podpisu

## Zadanie 5 (1 pkt.)

Spróbuj zaszyfrować plik bajka.txt za pomocą RSA. Co się dzieje?

## Zadanie 6 (1 pkt.)

Sprawdź, który z certyfikatów **ToJa1.crt**, **ToJa2.crt** czy **ToJa3.crt** jest prawdziwy. Certyfikat zaufanej trzeciej strony to **root.crt**.

Następnie wyekstrahuj klucz publiczny z prawdziwego certyfikatu i sprawdź, który z plików **plik1.txt**, **plik2.txt** czy **plik3.txt**, zostały podpisane przez właściciela certyfikatu. Podpisy plików znajdują się odpowiednio w plikach **signature1.bin**, **signature2.bin** i **signature3.bin**. Pliki podpisano za pomocą **SHA 1**.

## Zadanie 7 (2 pkt.)

W tym zadaniu przećwiczymy bardziej pełny schemat komunikacji. Stwórz trzy foldery **Alice**, **Bob**, **CA**. Alicja będzie nadawcą pliku, Bob odbiorcą a CA to zaufana trzecia strona.

Użyj następujących algorytmów:
- RSA 2048 e=65537
- SHA 256
- AES 256 CBC z kluczem generowanym z hasła

1. Każda ze stron generuje swoje pary klucz prywatny **privkey_Alice.pem** i klucz publiczny **pubkey_Alice.pem**.
2. W ogólości klucz prywatny należy zaszyfrować jakimś szyfrem symetrycznym, np. **AES** aby go bezpiecznie przechowywać, tu dla ułatwienia pomieniemy ten krok.
3. Zaufana trzecia strona **CA** tworzy swój certyfikat, podpisany przez samą siebie **root.crt**, który jest ogłaszany publicznie, tzn. trafia do Alicji i Boba.
4. Następnie Alicja i Bob muszą uzyskać certyfikaty podpisane przez **CA**. W związku z tym każde z nich tworzy rządanie certyfikatu **Alice_req.csr**.
5. Alicja i Bob wysyłają swoje żądania do **CA**, które po zweryfikowaniu Alicji i Boba podpisuje je. Podpisane certyfikaty Alicji i Boba sa ogłaszane publicznie.
6. Alicja bierze certyfikat Boba weryfikuje go za pomocą certyfikatu **CA** (**root.crt**). Jeśli certyfikat jest poprawny, wyciąga z niego klucz publiczny Boba i zapisuje do pliku **pubkey_Bob.pem**.
7. Alicja generuje losowe hasło (**sympass.pem**) z którego będzie generowany klucz symetryczny, które będzie chciała przekazać Bobowi.
8. Alicja szyfruje hasło **sympass.pem** za pomocą klucza publicznego Boba **pubkey_Bob.pem** otrzymując **sympass.enc**.
7. Alicja podpisuje zaszyfrowane hasło poprzez zhashowanie go i zaszyfrowanie za pomocą swojego klucza prywatnego **privkey_ALice.pem**. Podpis zapisuje w pliku **signature.bin**.
8. Alicja wysyła do Boba zaszyfrowane hasło **sympass.enc** oraz jego podpis **signature.bin**.
9. Bob weryfikuje certyfikat Alicji, wyciąga z niego jej klucz publiczny, deszyfruje otrzymane hasło **sympass.enc** i sprawdza podpis.
10. Alicja szyfruje plik, który chce przekazać Bobowi za pomocą hasła **sympass.pem** i wysyła zaszyfrowany plik do Boba.
11. Bob deszyfruje otrzymany plik za pomocą **sympass.pem**.